In [1]:
import numpy as np
import pandas as pd
from BCaOLSpy import BiasCorrectedOLS


- The regression equation is
$$y_i = \beta_x x_i + \beta_z z_i + u_i$$
with $\beta_x = 1$, $\beta_z = -1$, 
- $x_i, z_i$ jointly normal with mean zero, covariance 1, and correlation 0.3. 
- Errors are lognormally distributed with $\mu=0$ and $\sigma = 0.5$.
The location is shifted by subtracting $e^{\sigma^2/2}$ so that they are mean zero. 

In [2]:
np.random.seed(21)

# Simulating x and z
n = 50
rho = 0.3
var_xz = 1

mean = [0, 0]
cov = [[var_xz, rho * var_xz], [rho * var_xz, var_xz]]
x, z = np.random.multivariate_normal(mean, cov, n).T
beta_x, beta_z = 1.0, -1.0

In [3]:
# generating DGP for y
mu = 0
sigma = 0.5
u = np.random.lognormal(mean=mu, sigma=sigma, size=n) - np.exp(sigma**2/2)
y = beta_x * x + beta_z * z + u

In [4]:
# Create a DataFrame
df_example = pd.DataFrame({'y': y, 'x': x, 'z': z})


In [5]:
# Initialize and run analysis
bc_ols_example = BiasCorrectedOLS(df=df_example, dependent_var='y', independent_vars=['x', 'z'], verbose=True)
ols_coefs, ols_conf_int, _ = bc_ols_example.run_regression()
bc_ols_example.perform_bootstrap(n_sim=1_000)
bc_ols_example.perform_jackknife()

# Calculate BCa CI for each coefficient
beta_hat_x, bca_x, bca_ci_x = bc_ols_example.bca_estimate(ols_coefs['x'], bc_ols_example.bootstrap_distribution['x'], bc_ols_example.jackknife_distribution['x'])
beta_hat_z, bca_z, bca_ci_z = bc_ols_example.bca_estimate(ols_coefs['z'], bc_ols_example.bootstrap_distribution['z'], bc_ols_example.jackknife_distribution['z'])

Jackknife running...: 100%|██████████| 50/50 [00:00<00:00, 129.03it/s]


In [6]:
# Display the results
{
    "Regular OLS Coefficients": ols_coefs,
    "Regular OLS CI for x": ols_conf_int.loc['x'].tolist(),
    "Regular OLS CI for z": ols_conf_int.loc['z'].tolist(),
    "Original OLS beta for x": beta_hat_x,
    "Bias Corrected Estimate for x": bca_x,
    "BCa CI for x": bca_ci_x,
    "Original OLS beta for z": beta_hat_z,
    "Bias Corrected Estimate for z": bca_z,
    "BCa CI for z": bca_ci_z
}

{'Regular OLS Coefficients': Intercept   -0.160220
 x            1.021542
 z           -0.899324
 dtype: float64,
 'Regular OLS CI for x': [0.9070654269717732, 1.1360180518845857],
 'Regular OLS CI for z': [-0.9946313174258888, -0.8040162839862101],
 'Original OLS beta for x': 1.0215417394281794,
 'Bias Corrected Estimate for x': 1.0225383791442992,
 'BCa CI for x': (0.894544602449739, 1.139566033148775),
 'Original OLS beta for z': -0.8993238007060494,
 'Bias Corrected Estimate for z': -0.899449708786618,
 'BCa CI for z': (-1.0108064813087114, -0.8132203371969363)}